## 02. Creating a Major Merger (MM) catalogue to study AGN incidence due to galaxy mergers

The notebook is divided into the following sections:
1. Loading data and defining input parameters
2. Studying sizes of haloes and sub-haloes

**Script written by**: Soumya Shreeram <br>
**Project supervised by**: Johan Comparat <br>
**Date**: 30th March 2021

Import required packages

In [4]:
# scipy modules
from scipy.spatial import KDTree
from scipy.interpolate import interp1d

# astropy modules
import astropy.units as u
import astropy.io.fits as fits
from astropy.table import Table, Column
from astropy.coordinates import SkyCoord
from astropy.cosmology import FlatLambdaCDM, z_at_value

import numpy as np

# system imports
import os
import sys
import importlib as ib

# plotting imports
import matplotlib

Load the imported file(s) that contains all the functions used in this notebooks

In [5]:
%matplotlib inline
font = {'family' : 'serif',
        'weight' : 'medium',
        'size'   : 20}
matplotlib.rc('font', **font)

In [ ]:
sys.path.append('imported_files/')
import Exploring_DM_Haloes as edh
import Agn_incidence_from_Major_Mergers as aimm
import plotting_aimm02 as pt